# Lexical tokenization - Better TF\*IDF

Let's walk through a basic introduction to lexical search.

### Who you are:

An ML engineer with enough comfort with Python data stack (pandas, numpy, etc) that wants to understand traditional search engines (ie Elasticsearch, etc)

### What this is

A run through of the core concepts behind lexical search.


## This notebook: Better TFIDF (hint maybe its name begins with B)?

We [previously walked through text similarity scoring](https://colab.research.google.com/drive/1MOUa7u6kE_BWJEeueWjRuClctVxaJlJL#scrollTo=UevYFMMZmbp9). We used a first pass TF\*IDF formulation. But we'll see how that has problems, and how it can be improved.

In [ ]:
!pip install searcharray pystemmer

from searcharray import SearchArray
import pandas as pd
import numpy as np
import Stemmer


## Tokenize and index

Tokenize and index two fields:

1. The name (who's chatting)
2. Their message

**Note this time** we've added some longer messages

In [ ]:
from string import punctuation
stemmer = Stemmer.Stemmer('english')


def even_better_tokenize(text):
    lowercased = text.lower()
    without_punctuation = lowercased.translate(str.maketrans('', '', punctuation))
    split = without_punctuation.split()
    return [stemmer.stemWord(tok) for tok in split]


chat_transcript = [
  "Hi this is Doug, I have a complaint about the weather. My Doug Day is not Doug-tastic.",

  """
    Doug, we see you're having an issue with the climate. Doug, maybe you'd like to talk to the manager?
    Doug I think that'd be wise. What do you think Doug?
  """,
  "Tom, can I speak to your manager?",
  "Hi, this is Sue, Tom's boss. What can I do for you?",
  "I have complaints about the ski conditions in West Virginia",
  "Oh doug thats terrible, lets see what we can do."
]

msgs = pd.DataFrame({"name": ["Doug", "Doug", "Tom", "Sue", "Doug", "Sue"],
                     "msg": chat_transcript})
msgs['msg_tokenized'] = SearchArray.index(msgs['msg'],
                                          tokenizer=even_better_tokenize)
msgs['name_tokenized'] = SearchArray.index(msgs['name'],
                                          tokenizer=even_better_tokenize)
msgs

2025-06-14 21:10:19,787 - searcharray.indexing - INFO - Indexing begins w/ 4 workers


INFO:searcharray.indexing:Indexing begins w/ 4 workers


2025-06-14 21:10:19,795 - searcharray.indexing - INFO - 0 Batch Start tokenization


INFO:searcharray.indexing:0 Batch Start tokenization


2025-06-14 21:10:19,797 - searcharray.indexing - INFO - Tokenizing 6 documents


...

2025-06-14 21:10:19,835 - searcharray.indexing - INFO - (main thread) Processing 1 batch results


INFO:searcharray.indexing:(main thread) Processing 1 batch results


2025-06-14 21:10:19,837 - searcharray.indexing - INFO - Indexing from tokenization complete


INFO:searcharray.indexing:Indexing from tokenization complete


,name,msg,msg_tokenized,name_tokenized
0,Doug,"Hi this is Doug, I have a complaint about the ...","Terms({'hi', 'complaint', 'this', 'about', 'my...",Terms({'doug'})
1,Doug,"\n Doug, we see you're having an issue with...","Terms({'thatd', 'mayb', 'be', 'with', 'what', ...",Terms({'doug'})
2,Tom,"Tom, can I speak to your manager?","Terms({'your', 'manag', 'speak', 'i', 'can', '...",Terms({'tom'})
3,Sue,"Hi, this is Sue, Tom's boss. What can I do for...","Terms({'sue', 'you', 'hi', 'do', 'this', 'what...",Terms({'sue'})
4,Doug,I have complaints about the ski conditions in ...,"Terms({'virginia', 'complaint', 'condit', 'abo...",Terms({'doug'})
5,Sue,"Oh doug thats terrible, lets see what we can do.","Terms({'terribl', 'see', 'do', 'what', 'let', ...",Terms({'sue'})


## Search again w/ TF\*IDF

Recall we created a naive TF\*IDF similarity function last time. Let's use that!

In [ ]:
from searcharray.similarity import Similarity

def tf_idf(term_freqs: np.ndarray,        # TF array of every doc in the index
               doc_freqs: np.ndarray,         # Doc freq array of every term (> 1 if a phrase)
               doc_lens: np.ndarray,          # Every documents length (same shape as TF)
               avg_doc_lens: int,             # avg doc length of corpus
               num_docs: int) -> np.ndarray:     # total number of docs in corpus

    return term_freqs / (doc_freqs + 1)


In [ ]:
QUERY = "doug complaint"
FIELDS = ["msg_tokenized", "name_tokenized"]
query_tokenized = even_better_tokenize(QUERY)

# ACCUMULATE SCORES
scores = np.zeros(len(msgs))
for query_token in query_tokenized:
    # PASS SIMILARITY
    field_scores = np.zeros(len(msgs))
    for field in FIELDS:
        score = msgs[field].array.score(query_token,
                                        similarity=tf_idf)
        # Take maximum between field_scores and this field's score
        print(f"Field {field}, Term '{query_token}' score: {score}")
        field_scores = np.maximum(field_scores, score)
    print(f"Term '{query_token}' score: {field_scores}")
    scores += field_scores
    print(f"Scores now: {field_scores}")


msgs['scores'] = scores
msgs.sort_values('scores', ascending=False)

Field msg_tokenized, Term 'doug' score: [0.5  1.   0.   0.   0.   0.25]
Field name_tokenized, Term 'doug' score: [0.25 0.25 0.   0.   0.25 0.  ]
Term 'doug' score: [0.5  1.   0.   0.   0.25 0.25]
Scores now: [0.5  1.   0.   0.   0.25 0.25]
Field msg_tokenized, Term 'complaint' score: [0.33333333 0.         0.         0.         0.33333333 0.        ]
Field name_tokenized, Term 'complaint' score: [0. 0. 0. 0. 0. 0.]
Term 'complaint' score: [0.33333333 0.         0.         0.         0.33333333 0.        ]
Scores now: [0.33333333 0.         0.         0.         0.33333333 0.        ]


,name,msg,msg_tokenized,name_tokenized,scores
1,Doug,"\n Doug, we see you're having an issue with...","Terms({'thatd', 'mayb', 'be', 'with', 'what', ...",Terms({'doug'}),1.000000
0,Doug,"Hi this is Doug, I have a complaint about the ...","Terms({'hi', 'complaint', 'this', 'about', 'my...",Terms({'doug'}),0.833333
4,Doug,I have complaints about the ski conditions in ...,"Terms({'virginia', 'complaint', 'condit', 'abo...",Terms({'doug'}),0.583333
5,Sue,"Oh doug thats terrible, lets see what we can do.","Terms({'terribl', 'see', 'do', 'what', 'let', ...",Terms({'sue'}),0.250000
3,Sue,"Hi, this is Sue, Tom's boss. What can I do for...","Terms({'sue', 'you', 'hi', 'do', 'this', 'what...",Terms({'sue'}),0.000000
2,Tom,"Tom, can I speak to your manager?","Terms({'your', 'manag', 'speak', 'i', 'can', '...",Terms({'tom'}),0.000000


## Problem - Term frequency saturation

**Problem** - we notice this one document that says "Doug" a lot gets a high score. Obviously it has a high term frequency, but that doesn't automatically mean its more relevant _to our users_

Early Information Researchers realized, we don't want to take _raw_ term frequency, because after a while, it doesn't 'add' signal to the score. The core concept is **aboutness** - is this document _about_ the term? After a point saying "doug" over and over doesn't make it any more 'about' Doug.



### Take log of term freq?

Let's try a simple saturation, taking the log of term freq.

In [ ]:
from searcharray.similarity import Similarity

def tf_idf_saturate(term_freqs: np.ndarray,        # TF array of every doc in the index
                    doc_freqs: np.ndarray,         # Doc freq array of every term (> 1 if a phrase)
                    doc_lens: np.ndarray,          # Every documents length (same shape as TF)
                    avg_doc_lens: int,             # avg doc length of corpus
                    num_docs: int) -> np.ndarray:     # total number of docs in corpus

    return np.log(term_freqs) / (doc_freqs + 1) # TAKE LOG



QUERY = "doug complaint"
FIELDS = ["msg_tokenized", "name_tokenized"]
query_tokenized = even_better_tokenize(QUERY)

scores = np.zeros(len(msgs))
for query_token in query_tokenized:
    field_scores = np.zeros(len(msgs))
    for field in FIELDS:
        score = msgs[field].array.score(query_token,
                                        similarity=tf_idf_saturate) # CHANGED
        print(f"Field {field}, Term '{query_token}' score: {score}")
        field_scores = np.maximum(field_scores, score)
    print(f"Term '{query_token}' score: {field_scores}")
    scores += field_scores
    print(f"Scores now: {field_scores}")


msgs['scores'] = scores
msgs.sort_values('scores', ascending=False)

Field msg_tokenized, Term 'doug' score: [0.1732868  0.34657359       -inf       -inf       -inf 0.        ]
Field name_tokenized, Term 'doug' score: [  0.   0. -inf -inf   0. -inf]
Term 'doug' score: [0.1732868  0.34657359 0.         0.         0.         0.        ]
Scores now: [0.1732868  0.34657359 0.         0.         0.         0.        ]
Field msg_tokenized, Term 'complaint' score: [  0. -inf -inf -inf   0. -inf]
Field name_tokenized, Term 'complaint' score: [-inf -inf -inf -inf -inf -inf]
Term 'complaint' score: [0. 0. 0. 0. 0. 0.]
Scores now: [0. 0. 0. 0. 0. 0.]


<ipython-input-15-889925026>:9: RuntimeWarning: divide by zero encountered in log
  return np.log(term_freqs) / (doc_freqs + 1) # TAKE LOG


,name,msg,msg_tokenized,name_tokenized,scores
1,Doug,"\n Doug, we see you're having an issue with...","Terms({'thatd', 'mayb', 'be', 'with', 'what', ...",Terms({'doug'}),0.346574
0,Doug,"Hi this is Doug, I have a complaint about the ...","Terms({'hi', 'complaint', 'this', 'about', 'my...",Terms({'doug'}),0.173287
2,Tom,"Tom, can I speak to your manager?","Terms({'your', 'manag', 'speak', 'i', 'can', '...",Terms({'tom'}),0.000000
3,Sue,"Hi, this is Sue, Tom's boss. What can I do for...","Terms({'sue', 'you', 'hi', 'do', 'this', 'what...",Terms({'sue'}),0.000000
4,Doug,I have complaints about the ski conditions in ...,"Terms({'virginia', 'complaint', 'condit', 'abo...",Terms({'doug'}),0.000000
5,Sue,"Oh doug thats terrible, lets see what we can do.","Terms({'terribl', 'see', 'do', 'what', 'let', ...",Terms({'sue'}),0.000000


## Problem - field length

It's a bit better, but another bias that early IR researchers realized happens in text: a single term matching in shorter text matters more than longer text. A tweet mentioning a word once _is very much about that word_ OTOH a book mentioning a word once _is very much NOT about that word_.

In [ ]:
from searcharray.similarity import Similarity

def tf_idf_saturate_by_len(term_freqs: np.ndarray,        # TF array of every doc in the index
                           doc_freqs: np.ndarray,         # Doc freq array of every term (> 1 if a phrase)
                           doc_lens: np.ndarray,          # Every documents length (same shape as TF)
                           avg_doc_lens: int,             # avg doc length of corpus
                           num_docs: int) -> np.ndarray:     # total number of docs in corpus

    tf_idf_sat = np.log(term_freqs) / (doc_freqs + 1) # TAKE LOG
    tf_idf_sat /= (doc_lens + 1)
    return tf_idf_sat

QUERY = "doug complaint"
FIELDS = ["msg_tokenized", "name_tokenized"]
query_tokenized = even_better_tokenize(QUERY)

scores = np.zeros(len(msgs))
for query_token in query_tokenized:
    field_scores = np.zeros(len(msgs))
    for field in FIELDS:
        score = msgs[field].array.score(query_token,
                                        similarity=tf_idf_saturate_by_len) # CHANGED
        print(f"Field {field}, Term '{query_token}' score: {score}")
        field_scores = np.maximum(field_scores, score)
    print(f"Term '{query_token}' score: {field_scores}")
    scores += field_scores
    print(f"Scores now: {field_scores}")


msgs['scores'] = scores
msgs.sort_values('scores', ascending=False)

Field msg_tokenized, Term 'doug' score: [0.00962704 0.01117979       -inf       -inf       -inf 0.        ]
Field name_tokenized, Term 'doug' score: [  0.   0. -inf -inf   0. -inf]
Term 'doug' score: [0.00962704 0.01117979 0.         0.         0.         0.        ]
Scores now: [0.00962704 0.01117979 0.         0.         0.         0.        ]
Field msg_tokenized, Term 'complaint' score: [  0. -inf -inf -inf   0. -inf]
Field name_tokenized, Term 'complaint' score: [-inf -inf -inf -inf -inf -inf]
Term 'complaint' score: [0. 0. 0. 0. 0. 0.]
Scores now: [0. 0. 0. 0. 0. 0.]


<ipython-input-20-591062475>:9: RuntimeWarning: divide by zero encountered in log
  tf_idf_sat = np.log(term_freqs) / (doc_freqs + 1) # TAKE LOG


,name,msg,msg_tokenized,name_tokenized,scores
1,Doug,"\n Doug, we see you're having an issue with...","Terms({'thatd', 'mayb', 'be', 'with', 'what', ...",Terms({'doug'}),0.011180
0,Doug,"Hi this is Doug, I have a complaint about the ...","Terms({'hi', 'complaint', 'this', 'about', 'my...",Terms({'doug'}),0.009627
2,Tom,"Tom, can I speak to your manager?","Terms({'your', 'manag', 'speak', 'i', 'can', '...",Terms({'tom'}),0.000000
3,Sue,"Hi, this is Sue, Tom's boss. What can I do for...","Terms({'sue', 'you', 'hi', 'do', 'this', 'what...",Terms({'sue'}),0.000000
4,Doug,I have complaints about the ski conditions in ...,"Terms({'virginia', 'complaint', 'condit', 'abo...",Terms({'doug'}),0.000000
5,Sue,"Oh doug thats terrible, lets see what we can do.","Terms({'terribl', 'see', 'do', 'what', 'let', ...",Terms({'sue'}),0.000000


## BM25 - tuned TF\*IDF to the gills

BM25 (Best Match 25, as in 25th iteration) is the result of many iterations on using these statistics to score relevance.

It's innovation: it combines term frequency + doc length accordingly:

```
tf = term_freqs / (term_freqs + k1 * (1 - b + b * doc_lens / avg_doc_lens))
```

With two constants:

* k1 - controls how fast term freq saturation occurs
* b - ranged 0 to 1, controls how much longer field length will reduce the score

You can construct a raw similarity and play with it here, but its easier to [look at graphs](https://www.desmos.com/calculator/lukbszx5oe) to see the impact of these factors.

## Breadcrumbs for Elasticsearch, Vespa etc

BM25 is the default scoring function in Elasticsearch and Vespa